In [1]:
from google.colab import drive
drive.mount('/content/drive')
import shutil

source_path = '/content/drive/My Drive/ILSVRC2012_img_val.tar'

destination_path = '/content/ILSVRC2012_img_val.tar'

shutil.copy(source_path, destination_path)

Mounted at /content/drive


'/content/ILSVRC2012_img_val.tar'

In [2]:
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz

!tar xzf ILSVRC2012_devkit_t12.tar.gz

--2024-07-03 07:02:09--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2568145 (2.4M) [application/x-gzip]
Saving to: ‘ILSVRC2012_devkit_t12.tar.gz’

ILSVRC2012_devkit_t 100%[===================>]   2.45M  1.81MB/s    in 1.4s    

2024-07-03 07:02:11 (1.81 MB/s) - ‘ILSVRC2012_devkit_t12.tar.gz’ saved [2568145/2568145]



In [3]:
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from torch import nn


transforms = transforms.Compose([
    transforms.Resize((232,232)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [4]:
val_data = torchvision.datasets.ImageNet(root="/content", split='val', transform=transforms )


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]


In [7]:
!pip install wandb -qU
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
config = {
    'model': 'resnet50',
    'batch_size': 64
}


In [13]:
project_name = 'Transfer Learning Experiments'
run_name = 'Imagenet-Val'
github_url_base = 'https://github.com/Aadharsh1/ML-Deep-Learning/blob/main/Weights_Biases'
file_name = f'{run_name}.ipynb'
notebook_link = f'{github_url_base}/{project_name}/{file_name}'

run = wandb.init(
    project=project_name,
    name=run_name,
    notes=notebook_link,
    config=config,
    job_type="validate")

wandb: Currently logged in as: aadharshr-2022 (aadharshr-2022-Singapore Management University). Use `wandb login --relogin` to force relogin


In [9]:
calib_size = int(0.05 * len(val_data))
remaining_size = len(val_data) - calib_size

calib_set, val_set = random_split(val_data, [calib_size, remaining_size])
print(len(calib_set))

2500


In [14]:
val_loader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=False)

In [11]:
loss_fn = nn.CrossEntropyLoss()

def accuracy_function(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [15]:
from tqdm.auto import tqdm

val_loss, val_acc = 0, 0
resnet50.eval()
with torch.inference_mode():
  for images, labels in tqdm(val_loader):
    images, labels = images.to(device), labels.to(device)
    val_logits = resnet50(images)
    loss = loss_fn(val_logits, labels)
    val_loss += loss.item()
    val_acc += accuracy_function(labels, torch.argmax(val_logits, dim=1))
    wandb.log({"val_loss": loss.item(), "val_acc": accuracy_function(labels, torch.argmax(val_logits, dim=1))})
print(val_loss/len(val_loader), val_acc/len(val_loader))



  0%|          | 0/743 [00:00<?, ?it/s]

1.4112120878808752 80.37236428891879


In [ ]:
project_name = wandb.run.project
run_name = wandb.run.name
# change notebook name according to name of current notebook
notebook_path = '/content/drive/MyDrive/Colab Notebooks/imagenet_val_new.ipynb'